In [13]:
!pip install monai
!pip install einops

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from einops import rearrange

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [16]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
class ModularCNN(nn.Module):
    def __init__(self, n_classes: int):
        super(ModularCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.Dropout(p=0.4, inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.Dropout(p=0.4, inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1), 
            nn.Dropout(p=0.4, inplace=True), 
            nn.Conv2d(128, 128, kernel_size=3, padding=1), 
            nn.Dropout(p=0.4, inplace=True), 
            nn.Conv2d(128, 128, kernel_size=3, padding=1), 
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1), 
            nn.Dropout(p=0.4, inplace=True), 
            nn.Conv2d(256, 256, kernel_size=3, padding=1)
        )
        self.act = nn.Softmax(dim=1)
        self.dropout=nn.Dropout(p=0.4, inplace=True)
        self.fc1 = nn.Linear(128 * 16 * 16, 4096)
        self.fc2 = nn.Linear(4096, n_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        out = self.layer2(self.layer1(x))
        out = self.dropout(out)
        out = self.pool(out)
        out = rearrange(out, 'b c h w -> b (c h w)')
        out = self.relu(self.fc2(self.fc1(out)))
        return out

In [29]:
model = ModularCNN(n_classes = 10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [19]:
from tqdm import tqdm

def main(train_loader, test_loader, num_epochs):
    for epoch in range(num_epochs):
        train_loss, train_acc = 0, 0
        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            _, pred = torch.max(output, 1)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_acc += torch.sum(pred == labels.data)
        
        epoch_loss = train_loss / len(train_dataset)
        epoch_acc = (train_acc.double() / len(train_dataset)) * 100
        print(f"Epoch {epoch+1}/{num_epochs} Loss: {epoch_loss:.3f} Acc: {epoch_acc:.2f}%")
            

    with torch.no_grad():
        run_loss, run_acc = 0, 0
        for inputs, labels in tqdm(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            output = model(inputs)
            loss = criterion(output, labels)
            _, pred = torch.max(output, 1)
            run_loss += loss.item() * inputs.size(0)
            run_acc += torch.sum(pred == labels.data)
    test_loss = run_loss / len(test_dataset)
    test_acc = (run_acc.double() / len(test_dataset)) * 100
    print(f"Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%")

In [27]:
n_epochs: int = 10
main(train_loader, test_loader, n_epochs)

100%|██████████| 782/782 [00:33<00:00, 23.56it/s]


Epoch 1/10 Loss: 1.705 Acc: 39.69%


100%|██████████| 782/782 [00:33<00:00, 23.58it/s]


Epoch 2/10 Loss: 1.449 Acc: 48.68%


100%|██████████| 782/782 [00:33<00:00, 23.57it/s]


...

Epoch 9/10 Loss: 0.957 Acc: 66.97%


100%|██████████| 782/782 [00:33<00:00, 23.60it/s]


Epoch 10/10 Loss: 0.931 Acc: 67.87%


100%|██████████| 157/157 [00:02<00:00, 55.34it/s]

Test Loss: 1.042, Test Acc: 64.49%
